In [1]:
# Dependencies
from scipy.stats import poisson 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf


In [2]:
#register for api token at the-odds-api.com
#since odds are constantly updated, no guarantee will produce exact same results, look through before running again
TOKEN = "26b401b3192a94ed52c9e1dccd507c8a"

In [3]:
url="https://api.the-odds-api.com/v4/sports/soccer_epl/odds/?apiKey="+TOKEN+"&regions=uk&markets=h2h"

In [4]:
df =pd.read_json(url)

In [5]:
fixtures=df[["home_team","away_team","bookmakers"]]

In [6]:
data=fixtures.explode('bookmakers')

In [7]:
data["odds"]=data["bookmakers"].map(lambda x:x['markets'][0]["outcomes"])

In [8]:
data["bookmakers"]=data["bookmakers"].map(lambda x:x['key'])


In [9]:
data = pd.concat([data.reset_index(drop=True),pd.DataFrame(data["odds"].to_list(),columns=["bet_1","bet_2","bet_3"])],axis=1).drop("odds",axis=1)

In [10]:
data

,home_team,away_team,bookmakers,bet_1,bet_2,bet_3
0,Tottenham Hotspur,Everton,paddypower,"{'name': 'Everton', 'price': 5.5}","{'name': 'Tottenham Hotspur', 'price': 1.62}","{'name': 'Draw', 'price': 3.8}"
1,Tottenham Hotspur,Everton,betclic,"{'name': 'Everton', 'price': 5.15}","{'name': 'Tottenham Hotspur', 'price': 1.67}","{'name': 'Draw', 'price': 4.05}"
2,Tottenham Hotspur,Everton,livescorebet,"{'name': 'Everton', 'price': 5.5}","{'name': 'Tottenham Hotspur', 'price': 1.67}","{'name': 'Draw', 'price': 4.0}"
3,Tottenham Hotspur,Everton,virginbet,"{'name': 'Everton', 'price': 5.5}","{'name': 'Tottenham Hotspur', 'price': 1.67}","{'name': 'Draw', 'price': 4.0}"
4,Tottenham Hotspur,Everton,marathonbet,"{'name': 'Everton', 'price': 5.65}","{'name': 'Tottenham Hotspur', 'price': 1.64000...","{'name': 'Draw', 'price': 4.2}"
...,...,...,...,...,...,...
357,Tottenham Hotspur,West Ham United,livescorebet,"{'name': 'Tottenham Hotspur', 'price': 2.15}","{'name': 'West Ham United', 'price': 3.25}","{'name': 'Draw', 'price': 3.6}"
358,Tottenham Hotspur,West Ham United,ladbrokes,"{'name': 'Tottenham Hotspur', 'price': 2.15}","{'name': 'West Ham United', 'price': 3.25}","{'name': 'Draw', 'price': 3.45}"
359,Tottenham Hotspur,West Ham United,coral,"{'name': 'Tottenham Hotspur', 'price': 2.15}","{'name': 'West Ham United', 'price': 3.2}","{'name': 'Draw', 'price': 3.4}"
360,Tottenham Hotspur,West Ham United,betvictor,"{'name': 'Tottenham Hotspur', 'price': 2.15}","{'name': 'West Ham United', 'price': 3.2}","{'name': 'Draw', 'price': 3.4}"


In [11]:
def convert_home_unfair(row):
    if row["bet_1"]["name"]==row["home_team"]:
        return row["bet_1"]["price"]
    elif row["bet_2"]["name"]==row["home_team"]:
        return row["bet_2"]["price"]
    elif row["bet_3"]["name"]==row["home_team"]: 
        return row["bet_3"]["price"]
def convert_away_unfair(row):
    if row["bet_1"]["name"]==row["away_team"]:
        return row["bet_1"]["price"]
    elif row["bet_2"]["name"]==row["away_team"]:
        return row["bet_2"]["price"]
    elif row["bet_3"]["name"]==row["away_team"]:
        return row["bet_3"]["price"]
def convert_draw_unfair(row):
    if row["bet_3"]["name"]=="Draw":
        return row["bet_3"]["price"]
    elif row["bet_1"]["name"]=="Draw":
        return row["bet_1"]["price"]
    elif row["bet_2"]["name"]=="Draw": 
        return row["bet_2"]["price"]

In [12]:
data["home_unfair"]=data.apply(convert_home_unfair,axis=1)
data["away_unfair"]=data.apply(convert_away_unfair,axis=1)
data["draw_unfair"]=data.apply(convert_draw_unfair,axis=1)

In [13]:
data=data.drop(["bet_1","bet_2","bet_3"],axis=1)

In [14]:
data["overround"] = (1/data["home_unfair"])+(1/data["away_unfair"])+(1/data["draw_unfair"])

In [15]:
data["home_fair_prob"]=(1/data["home_unfair"])/data["overround"]
data["away_fair_prob"]=(1/data["away_unfair"])/data["overround"]
data["draw_fair_prob"]=(1/data["draw_unfair"])/data["overround"]
data

,home_team,away_team,bookmakers,home_unfair,away_unfair,draw_unfair,overround,home_fair_prob,away_fair_prob,draw_fair_prob
0,Tottenham Hotspur,Everton,paddypower,1.62,5.50,3.80,1.062260,0.581104,0.171162,0.247734
1,Tottenham Hotspur,Everton,betclic,1.67,5.15,4.05,1.039891,0.575832,0.186726,0.237442
2,Tottenham Hotspur,Everton,livescorebet,1.67,5.50,4.00,1.030621,0.581011,0.176416,0.242572
3,Tottenham Hotspur,Everton,virginbet,1.67,5.50,4.00,1.030621,0.581011,0.176416,0.242572
4,Tottenham Hotspur,Everton,marathonbet,1.64,5.65,4.20,1.024842,0.594975,0.172701,0.232324
...,...,...,...,...,...,...,...,...,...,...
357,Tottenham Hotspur,West Ham United,livescorebet,2.15,3.25,3.60,1.050586,0.442721,0.292877,0.264403
358,Tottenham Hotspur,West Ham United,ladbrokes,2.15,3.25,3.45,1.062664,0.437689,0.289548,0.272763
359,Tottenham Hotspur,West Ham United,coral,2.15,3.20,3.40,1.071734,0.433985,0.291584,0.274432
360,Tottenham Hotspur,West Ham United,betvictor,2.15,3.20,3.40,1.071734,0.433985,0.291584,0.274432


In [16]:
def calculate_real_home_bookmaker(row):
    home_team = row["home_team"]
    away_team = row["away_team"]
    m = data.groupby(['home_team','away_team'])[["home_fair_prob","away_fair_prob","draw_fair_prob"]].mean()
    mean_home = m["home_fair_prob"].loc[home_team,away_team]
    return mean_home
def calculate_real_away_bookmaker(row):
    home_team = row["home_team"]
    away_team = row["away_team"]
    m = data.groupby(['home_team','away_team'])[["home_fair_prob","away_fair_prob","draw_fair_prob"]].mean()
    mean_away=m["away_fair_prob"].loc[home_team,away_team]
    return mean_away
def calculate_real_draw_bookmaker(row):
    home_team = row["home_team"]
    away_team = row["away_team"]
    m = data.groupby(['home_team','away_team'])[["home_fair_prob","away_fair_prob","draw_fair_prob"]].mean()
    mean_draw = m["draw_fair_prob"].loc[home_team,away_team]
    return mean_draw

In [17]:
data["mean_home"] = data.apply(calculate_real_home_bookmaker,axis=1)
data["mean_away"] = data.apply(calculate_real_away_bookmaker,axis=1)
data["mean_draw"] = data.apply(calculate_real_draw_bookmaker,axis=1)

In [18]:
data["edge_home"] = (1/data["home_unfair"])/data["mean_home"]
data["edge_away"] = (1/data["away_unfair"])/data["mean_away"]
data["edge_draw"] = (1/data["draw_unfair"])/data["mean_draw"]
data

,home_team,away_team,bookmakers,home_unfair,away_unfair,draw_unfair,overround,home_fair_prob,away_fair_prob,draw_fair_prob,mean_home,mean_away,mean_draw,edge_home,edge_away,edge_draw
0,Tottenham Hotspur,Everton,paddypower,1.62,5.50,3.80,1.062260,0.581104,0.171162,0.247734,0.584995,0.174186,0.240819,1.055195,1.043818,1.092762
1,Tottenham Hotspur,Everton,betclic,1.67,5.15,4.05,1.039891,0.575832,0.186726,0.237442,0.584995,0.174186,0.240819,1.023602,1.114757,1.025308
2,Tottenham Hotspur,Everton,livescorebet,1.67,5.50,4.00,1.030621,0.581011,0.176416,0.242572,0.584995,0.174186,0.240819,1.023602,1.043818,1.038124
3,Tottenham Hotspur,Everton,virginbet,1.67,5.50,4.00,1.030621,0.581011,0.176416,0.242572,0.584995,0.174186,0.240819,1.023602,1.043818,1.038124
4,Tottenham Hotspur,Everton,marathonbet,1.64,5.65,4.20,1.024842,0.594975,0.172701,0.232324,0.584995,0.174186,0.240819,1.042326,1.016106,0.988690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,Tottenham Hotspur,West Ham United,livescorebet,2.15,3.25,3.60,1.050586,0.442721,0.292877,0.264403,0.443172,0.290218,0.266610,1.049516,1.060211,1.041889
358,Tottenham Hotspur,West Ham United,ladbrokes,2.15,3.25,3.45,1.062664,0.437689,0.289548,0.272763,0.443172,0.290218,0.266610,1.049516,1.060211,1.087189
359,Tottenham Hotspur,West Ham United,coral,2.15,3.20,3.40,1.071734,0.433985,0.291584,0.274432,0.443172,0.290218,0.266610,1.049516,1.076777,1.103177
360,Tottenham Hotspur,West Ham United,betvictor,2.15,3.20,3.40,1.071734,0.433985,0.291584,0.274432,0.443172,0.290218,0.266610,1.049516,1.076777,1.103177


In [19]:
best_home = data.iloc[data.groupby(['home_team','away_team'])['home_unfair'].idxmax()][["home_team","away_team","home_unfair","bookmakers"]]
best_away = data.iloc[data.groupby(['home_team','away_team'])['away_unfair'].idxmax()][["home_team","away_team","away_unfair","bookmakers"]]
best_draw = data.iloc[data.groupby(['home_team','away_team'])['draw_unfair'].idxmax()][["home_team","away_team","draw_unfair","bookmakers"]]
best_home

,home_team,away_team,home_unfair,bookmakers
247,Arsenal,Leicester City,1.53,marathonbet
297,Arsenal,Liverpool,4.20,paddypower
331,Aston Villa,Arsenal,3.75,paddypower
119,Brentford,Burnley,2.26,betfair
100,Brighton and Hove Albion,Liverpool,7.60,betfair
292,Brighton and Hove Albion,Tottenham Hotspur,3.10,coral
178,Chelsea,Newcastle United,1.34,betfair
273,Crystal Palace,Manchester City,11.50,betvictor
309,Everton,Newcastle United,2.26,betclic
192,Everton,Wolverhampton Wanderers,2.44,betclic


In [20]:
data.groupby(['home_team','away_team'])['home_unfair'].idxmax()

home_team                 away_team              
Arsenal                   Leicester City             247
                          Liverpool                  297
Aston Villa               Arsenal                    331
Brentford                 Burnley                    119
Brighton and Hove Albion  Liverpool                  100
                          Tottenham Hotspur          292
Chelsea                   Newcastle United           178
Crystal Palace            Manchester City            273
Everton                   Newcastle United           309
                          Wolverhampton Wanderers    192
Leeds United              Aston Villa                 81
                          Norwich City               209
Leicester City            Brentford                  340
Manchester United         Tottenham Hotspur          133
Norwich City              Chelsea                     25
Southampton               Newcastle United            38
                          Watford     

In [ ]:
best_odds = best_home.merge(best_away,on=['home_team','away_team']).merge(best_draw,on=['home_team','away_team'])
best_odds["overround_best"] = (1/best_odds["home_unfair"])+(1/best_odds["away_unfair"])+(1/best_odds["draw_unfair"])
best_odds

In [ ]:
def arbitrage(stake,home_unfair,away_unfair,draw_unfair):
        home_stake = stake*(1/home_unfair)
        away_stake = stake*(1/away_unfair)
        draw_stake = stake*(1/draw_unfair)
        home_profit = home_stake*(home_unfair-1)
        away_profit = away_stake*(away_unfair-1)
        draw_profit = draw_stake*(draw_unfair-1)
        lost_home = away_stake+draw_stake
        lost_away = home_stake+draw_stake
        lost_draw = home_stake+away_stake
        print('Home Stake:',home_stake,'Away Stake:',away_stake,'Draw Stake:',draw_stake)
        print('Home Profit:',home_profit,'Away Profit:',away_profit,'Draw Profit',draw_profit)
        print('Home Lost Stakes:',lost_home,'Away Lost Stakes:',lost_away,'Draw Lost Stakes:',lost_draw)
        print('Overall Profit Home:',home_profit-lost_home,'Overall Profit Away:',away_profit-lost_away,'Overall Profit Draw:',draw_profit-lost_draw)

In [ ]:
arbitrage(200,2.48,3.6,3.2)

In [ ]:
best_home_odds = data.groupby(['home_team','away_team'])['home_unfair'].max()
best_away_odds = data.groupby(['home_team','away_team'])['away_unfair'].max()
best_draw_odds = data.groupby(['home_team','away_team'])['draw_unfair'].max()
best_odds = pd.DataFrame({"best_home":best_home_odds,"best_away":best_away_odds,"best_draw":best_draw_odds})

In [ ]:
best_odds["overround"] = ((1/best_odds["best_home"])+(1/best_odds["best_draw"])+(1/best_odds["best_away"]))
best_odds